In [1]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data

mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)


X = tf.placeholder(tf.float32, [None, 784])
y_true = tf.placeholder(tf.float32, [None, 10])

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


__把加layer寫成一個function__

這編寫的add_layer其實只是幫我做了把input資料，乘上一個W矩陣加上bias，也就是一層fully connected層的意思，最後activation function再經過一個softmax，最後輸出就是網路的輸出，已經可以和ground truth label算cross entropy

__NOTE: __裡面的 __W__ 和 __bias__ 宣告的方式會影響其初始化的方式，往往對訓練品質會有很大的影響，需要作好的調整

In [2]:
def add_layer(inputs, in_size, out_size, activation_function=None):
    """
    args1 input: 輸入input的placeholder
    args2 in_size: input維度
    args3 out_size: output維度
    args4 activation_function: 激勵函數，通常是tf.nn下的函式
    """
    W = tf.Variable(tf.zeros([in_size, out_size]))
    bias = tf.Variable(tf.zeros([out_size]) + 0.1)
    Wx_plus_b = tf.matmul(inputs, W) + bias
    if activation_function is None:
        outputs = Wx_plus_b
    else:
        outputs = activation_function(Wx_plus_b)
    return outputs

__ML第一步-定義模型__

這邊使用寫好的add_layer函式，使用softmax函式，輸入維度維784，輸出維度為10

In [ ]:
output = add_layer(X, 784, 10, activation_function=tf.nn.softmax)

ML第二步-定義Loss Function

In [ ]:
cross_entropy = tf.reduce_mean(-tf.reduce_sum(y_true * tf.log(output), reduction_indices=[1]))

ML第三步-Gradient Descent

In [ ]:
train_step = tf.train.GradientDescentOptimizer(0.5).minimize(cross_entropy)

--------

開始執行computational graph

In [ ]:
init = tf.global_variables_initializer()

In [ ]:
sess = tf.Session()

In [ ]:
sess.run(init)

In [ ]:
from time import time

In [ ]:
start = time()
for _ in range(1000):
    batch_xs, batch_ys = mnist.train.next_batch(100)
    sess.run(train_step, feed_dict={X: batch_xs, y_true: batch_ys})
end = time()
print('訓練花費', (end-start), '秒')

---

evaluation

In [ ]:
correct_prediction = tf.equal(tf.argmax(output, 1), tf.argmax(y_true, 1)) # 注意這邊的 y 和 y_ 都是placeholder
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
print('Accuracy: ', sess.run(accuracy, feed_dict = {X: mnist.test.images, y_true: mnist.test.labels}))
